In [174]:
%load_ext autoreload
%autoreload 2

In [175]:
from concurrent.futures import ThreadPoolExecutor
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

In [176]:
dataset_root_dir = r"C:\UCL Coursework\Sem 1\COMP0173\sustainbench\dhs" # '../../dataset_preprocessing/dhs_lsms'

In [177]:
# Satelite data
df = pd.read_csv(os.path.join(dataset_root_dir, 'dhs_final_labels.csv'))
df['survey'] = df['DHSID_EA'].str[:10]
df['cc'] = df['DHSID_EA'].str[:2]
# df['path'] = r'C:\UCL Coursework\Sem 1\COMP0173\sustainbench\dhs\dhs_AL_DR/'+ df['survey'] + '/' + df['DHSID_EA'] + '.npz'
df['path'] = dataset_root_dir + '/dhs_npzs/' + df['survey'] + '/' + df['DHSID_EA'] + '.npz'
path_years = df[['DHSID_EA', 'path', 'year']].apply(tuple, axis=1)
df.set_index('DHSID_EA', verify_integrity=True, inplace=True)
print(df['path'].iloc[0])
display(df.head())

C:\UCL Coursework\Sem 1\COMP0173\sustainbench\dhs/dhs_npzs/AL-2008-5#/AL-2008-5#-00000001.npz


,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,n_sanitation,sanitation_index,...,women_bmi,n_women_edu,n_women_bmi,cluster_id,adm1fips,adm1dhs,urban,survey,cc,path
DHSID_EA,,,,,,,,,,,,,,,,,,,,,
AL-2008-5#-00000001,AL,2008,40.822652,19.838321,18.0,2.430596,18.0,3.444444,18.0,4.833333,...,24.365000,18.0,18.0,1,NaN,9999,R,AL-2008-5#,AL,C:\UCL Coursework\Sem 1\COMP0173\sustainbench\...
AL-2008-5#-00000002,AL,2008,40.696846,20.007555,20.0,2.867678,20.0,4.700000,20.0,4.950000,...,23.104000,20.0,20.0,2,NaN,9999,R,AL-2008-5#,AL,C:\UCL Coursework\Sem 1\COMP0173\sustainbench\...
AL-2008-5#-00000003,AL,2008,40.750037,19.974262,18.0,2.909049,18.0,4.500000,18.0,4.777778,...,22.387778,18.0,18.0,3,NaN,9999,R,AL-2008-5#,AL,C:\UCL Coursework\Sem 1\COMP0173\sustainbench\...
AL-2008-5#-00000004,AL,2008,40.798931,19.863338,19.0,2.881122,19.0,4.947368,19.0,4.789474,...,27.084500,21.0,20.0,4,NaN,9999,R,AL-2008-5#,AL,C:\UCL Coursework\Sem 1\COMP0173\sustainbench\...
AL-2008-5#-00000005,AL,2008,40.746123,19.843885,19.0,2.546830,19.0,4.684211,19.0,4.526316,...,24.523125,16.0,16.0,5,NaN,9999,R,AL-2008-5#,AL,C:\UCL Coursework\Sem 1\COMP0173\sustainbench\...


In [178]:
label_cols = ['asset_index', 'under5_mort', 'women_bmi', 'women_edu', 'water_index', 'sanitation_index']

In [179]:
def calculate_nl_mean(path_and_year) -> tuple[np.ndarray, np.ndarray, int]:
    '''
    Args
    - path_year: tuple (path, year)
      - path: str, path to npz file containing single entry 'x'
        representing a (C, H, W) image
      - year: int

    Returns: (nl_mean, year)
    '''
    dhsid_ea, npz_path, year = path_and_year
    img = np.load(npz_path)['x']  # shape (C, H, W)
    nl_mean = img[-1].mean(dtype=np.float64)
    return dhsid_ea, nl_mean, year

In [180]:
results_df = pd.DataFrame(
    data=np.nan,
    columns=['nl_mean', 'year'],
    index=df.index #pd.Index(sorted(df['DHSID_EA']), name='DHSID_EA')
)
results_df.head()

,nl_mean,year
DHSID_EA,,
AL-2008-5#-00000001,NaN,NaN
AL-2008-5#-00000002,NaN,NaN
AL-2008-5#-00000003,NaN,NaN
AL-2008-5#-00000004,NaN,NaN
AL-2008-5#-00000005,NaN,NaN


In [181]:
# see current path
print(os.getcwd())
# make experiment result folder
os.makedirs('experiment_results', exist_ok=True)
# make subfolder for data exploration a
os.makedirs('experiment_results/data_exploration', exist_ok=True)

c:\UCL Coursework\Sem 1\COMP0173\sustainbench\baseline_models\dhs


In [ ]:
# These are street level data
import glob
metadata_path = r"C:/UCL Coursework/Sem 1/COMP0173/sustainbench/dhs/mapillary/metadata"
metadata_files = glob.glob(os.path.join(metadata_path, "*.csv"))
metadata = pd.concat([pd.read_csv(file) for file in metadata_files])
metadata.head()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
np.random.seed(0)
# choice random vector of int [0, 200] length 10
save_dir = "experiment_results/data_exploration"
choices = np.random.randint(0, len(path_years), 10)

for i, choice in enumerate(choices):
    dhsid_ea, npz_path, year = path_years[choice]
    img = np.load(npz_path)['x']  # shape (C, H, W) -> (8, 256, 256)

    fig, ax = plt.subplots(2, 3, figsize=(10, 8))
    ax = ax.flatten()

    # No ticks
    for a in ax:
        a.set_xticks([])
        a.set_yticks([])

    # RGB Bands
    img_rgb = img[:3].transpose(1, 2, 0)  # Select the first three bands for RGB
    img_rgb = img_rgb[..., ::-1]  # BGR to RGB
    img_rgb_normalized = (img_rgb - img_rgb.min()) / (img_rgb.max() - img_rgb.min())

    ax[0].imshow(img_rgb_normalized)
    ax[0].set_title('RGB Band', fontsize=14)

    # NIR Band
    ax[1].imshow(img[3], cmap='gray')
    ax[1].set_title('NIR Band', fontsize=14)

    # SWIR1 Band
    ax[2].imshow(img[4], cmap='gray')
    ax[2].set_title('SWIR1 Band', fontsize=14)

    # SWIR2 Band
    ax[3].imshow(img[5], cmap='gray')
    ax[3].text(0.5, -0.05, 'SWIR2 Band', transform=ax[3].transAxes, ha='center', va='top', fontsize=14)

    # TEMP1 Band
    ax[4].imshow(img[6], cmap='gray')
    ax[4].text(0.5, -0.05, 'TEMP1 Band', transform=ax[4].transAxes, ha='center', va='top', fontsize=14)

    # Nightlight Band
    ax[5].imshow(img[7], cmap='gray')
    ax[5].text(0.5, -0.05, 'Nightlight Band', transform=ax[5].transAxes, ha='center', va='top', fontsize=14)

    fig.suptitle(f'{dhsid_ea} - {year}', fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 1])
    plt.show()
    fig.savefig(f'{save_dir}/{i}.png')

In [185]:
with ThreadPoolExecutor(max_workers=30) as pool:
    inputs = path_years
    futures = pool.map(calculate_nl_mean, inputs)
    for dhsid_ea, nl_mean, year in tqdm(futures, total=len(inputs)):
        results_df.loc[dhsid_ea, ['nl_mean', 'year']] = (nl_mean, year)

  0%|          | 0/117644 [00:00<?, ?it/s]

In [186]:
results_df.to_csv('mean_nl.csv')

In [187]:
results_df['year'] = results_df['year'].astype(int)

In [188]:
SPLITS = {
    'train': [
        'AL', 'BD', 'CD', 'CM', 'GH', 'GU', 'HN', 'IA', 'ID', 'JO', 'KE', 'KM',
        'LB', 'LS', 'MA', 'MB', 'MD', 'MM', 'MW', 'MZ', 'NG', 'NI', 'PE', 'PH',
        'SN', 'TG', 'TJ', 'UG', 'ZM', 'ZW'],
    'val': [
        'BF', 'BJ', 'BO', 'CO', 'DR', 'GA', 'GN', 'GY', 'HT', 'NM', 'SL', 'TD',
        'TZ'],
    'test': [
        'AM', 'AO', 'BU', 'CI', 'EG', 'ET', 'KH', 'KY', 'ML', 'NP', 'PK', 'RW',
        'SZ']
}
SPLITS['trainval'] = SPLITS['train'] + SPLITS['val']

In [189]:
import scipy.stats
import sklearn.neighbors

In [190]:
results_df['cc'] = results_df.index.str[:2]

In [191]:
def run(knn, label, dmsp, trainsplit='train', testsplit='test'):
    if dmsp:
        year_mask = (df['year'] <= 2011)
    else:
        year_mask = (df['year'] > 2011)

    train_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[trainsplit]) & df[label].notna()]
    test_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[testsplit]) & df[label].notna()]

    train_X = results_df.loc[train_dhsids, 'nl_mean'].values.reshape(-1, 1)
    train_Y = df.loc[train_dhsids, label].values
    test_X = results_df.loc[test_dhsids, 'nl_mean'].values.reshape(-1, 1)
    test_Y = df.loc[test_dhsids, label].values

    knn.fit(train_X, train_Y)
    preds = knn.predict(test_X)
    return preds, test_Y

In [219]:
# Write for polynomial regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def run(model, label, dmsp, trainsplit='train', testsplit='test'):
    if dmsp:
        year_mask = (df['year'] <= 2011)
    else:
        year_mask = (df['year'] > 2011)

    train_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[trainsplit]) & df[label].notna()]
    test_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[testsplit]) & df[label].notna()]

    train_X = results_df.loc[train_dhsids, 'nl_mean'].values.reshape(-1, 1)
    train_Y = df.loc[train_dhsids, label].values
    test_X = results_df.loc[test_dhsids, 'nl_mean'].values.reshape(-1, 1)
    test_Y = df.loc[test_dhsids, label].values

    model.fit(train_X, train_Y)
    preds = model.predict(test_X)
    return preds, test_Y

poly= make_pipeline(PolynomialFeatures(2), LinearRegression())
preds_dmsp, labels_dmsp = run(poly, label, True, 'train', 'val')
preds_viirs, labels_viirs = run(poly, label, False, 'train', 'val')
r2 = scipy.stats.pearsonr(
    np.concatenate([preds_dmsp, preds_viirs]),
    np.concatenate([labels_dmsp, labels_viirs])
)[0]**2
print(f'k={k:2d}, {label:15s} r^2 = {r2:.3f}')
if r2 > best_r2:
    best_r2 = r2
    best_k = k
    

k=20, under5_mort     r^2 = 0.003


In [207]:
for label in label_cols:
    print(f'=========== {label:15s} ============')
    best_r2 = 0
    best_k = None
    for k in range(1, 21):
        knn = sklearn.neighbors.KNeighborsRegressor(
            n_neighbors=k, weights='uniform', algorithm='auto')
        preds_dmsp, labels_dmsp = run(knn, label, True, 'train', 'val')
        preds_viirs, labels_viirs = run(knn, label, False, 'train', 'val')
        r2 = scipy.stats.pearsonr(
            np.concatenate([preds_dmsp, preds_viirs]),
            np.concatenate([labels_dmsp, labels_viirs])
        )[0]**2
        print(f'k={k:2d}, {label:15s} r^2 = {r2:.3f}')
        if r2 > best_r2:
            best_r2 = r2
            best_k = k
    knn = sklearn.neighbors.KNeighborsRegressor(
            n_neighbors=best_k, weights='uniform', algorithm='auto')
    preds_dmsp, labels_dmsp = run(knn, label, True, 'trainval', 'test')
    preds_viirs, labels_viirs = run(knn, label, False, 'trainval', 'test')
    r2 = scipy.stats.pearsonr(
        np.concatenate([preds_dmsp, preds_viirs]),
        np.concatenate([labels_dmsp, labels_viirs])
    )[0]**2
    print(f'FINAL: k={best_k:2d}, {label:15s} r^2 = {r2:.2f}')

=========== asset_index     ============
k= 1, asset_index     r^2 = 0.266
k= 2, asset_index     r^2 = 0.322
k= 3, asset_index     r^2 = 0.364
k= 4, asset_index     r^2 = 0.395
k= 5, asset_index     r^2 = 0.386
k= 6, asset_index     r^2 = 0.382
k= 7, asset_index     r^2 = 0.382
k= 8, asset_index     r^2 = 0.391
k= 9, asset_index     r^2 = 0.390
k=10, asset_index     r^2 = 0.395
k=11, asset_index     r^2 = 0.393
k=12, asset_index     r^2 = 0.399
k=13, asset_index     r^2 = 0.402
k=14, asset_index     r^2 = 0.397
k=15, asset_index     r^2 = 0.397
k=16, asset_index     r^2 = 0.400
k=17, asset_index     r^2 = 0.393
k=18, asset_index     r^2 = 0.390
k=19, asset_index     r^2 = 0.387
k=20, asset_index     r^2 = 0.383
FINAL: k=13, asset_index     r^2 = 0.43
=========== under5_mort     ============
k= 1, under5_mort     r^2 = 0.000
k= 2, under5_mort     r^2 = 0.001
k= 3, under5_mort     r^2 = 0.000
k= 4, under5_mort     r^2 = 0.000
k= 5, under5_mort     r^2 = 0.000
k= 6, under5_mort     r^2 = 

In [ ]:
def run(model, label, dmsp, trainsplit='train', testsplit='test', plot=False):
    if dmsp:
        year_mask = (df['year'] <= 2011)
    else:
        year_mask = (df['year'] > 2011)

    train_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[trainsplit]) & df[label].notna()]
    test_dhsids = df.index[year_mask & df['cc'].isin(SPLITS[testsplit]) & df[label].notna()]

    train_X = results_df.loc[train_dhsids, 'nl_mean'].values.reshape(-1, 1)
    train_Y = df.loc[train_dhsids, label].values
    test_X = results_df.loc[test_dhsids, 'nl_mean'].values.reshape(-1, 1)
    test_Y = df.loc[test_dhsids, label].values

    model.fit(train_X, train_Y)
    preds = knn.predict(test_X)
    
    # plot predictions
    if plot:
        fig, ax = plt.subplots(1,2, figsize=(10, 5))
        ax = ax.flatten()
        # plot the knn results clustering, with the color representing the label
        ax[0].scatter(train_X, train_Y, alpha=0.5)
        ax[0].scatter(test_X, preds, alpha=0.5)
        ax[0].set_xlabel('Nightlight Mean')
        ax[0].set_ylabel(label)
        ax[0].set_title(f'{label} - {trainsplit} - {testsplit}')
        ax[0].legend(['Train', 'Test'])
        
        print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)
        # plot something that would show the r2 score
        ax[1].scatter(test_Y, preds, alpha=0.5)
        ax[1].plot(
            [test_Y.min(), test_Y.max()],
            [test_Y.min(), test_Y.max()], 
            'k--')
        ax[1].set_xlabel('True')
        ax[1].set_ylabel('Pred')
        ax[1].set_title(f'{label} - {trainsplit} - {testsplit}')
        fig.show()     
        
    return preds, test_Y

best_ks = 13, 17
for best_k, label in zip(best_ks, label_cols):
    print(f'=========== {label:15s} ============')
    knn = sklearn.neighbors.KNeighborsRegressor(
            n_neighbors=best_k, weights='uniform', algorithm='auto')
    preds_dmsp, labels_dmsp   = run(knn, label, True, 'trainval', 'test', plot=True)
    preds_viirs, labels_viirs = run(knn, label, False, 'trainval','test', plot=True)
    r2 = scipy.stats.pearsonr(
        np.concatenate([preds_dmsp, preds_viirs]),
        np.concatenate([labels_dmsp, labels_viirs])
    )[0]**2
    print(f'FINAL: k={best_k:2d}, {label:15s} r^2 = {r2:.2f}')
    
from sklearn.svm import SVR
svr_rbf = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=100, gamma="auto")
svr_poly = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
poly = make_pipeline(PolynomialFeatures(10), LinearRegression())    

# hyperparameter tuning

for model in [svr_rbf, svr_lin, svr_poly, poly]:
    for best_k, label in zip(best_ks, label_cols):
        print(f'=========== {label:15s} ============')
        preds_dmsp, labels_dmsp   = run(model, label, True, 'trainval', 'test', plot=True)
        preds_viirs, labels_viirs = run(model, label, False, 'trainval','test', plot=True)
        r2 = scipy.stats.pearsonr(
            np.concatenate([preds_dmsp, preds_viirs]),
            np.concatenate([labels_dmsp, labels_viirs])
        )[0]**2
        print(f'FINAL: k={best_k:2d}, {label:15s} r^2 = {r2:.2f}')



=========== asset_index     ============
(22960, 1) (22960,) (3201, 1) (3201,)


C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()
C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


(53433, 1) (53433,) (7342, 1) (7342,)
FINAL: k=13, asset_index     r^2 = 0.43
=========== under5_mort     ============
(34249, 1) (34249,) (12003, 1) (12003,)
(51865, 1) (51865,) (7465, 1) (7465,)
FINAL: k=17, under5_mort     r^2 = 0.00
=========== asset_index     ============


C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()
C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


(22960, 1) (22960,) (3201, 1) (3201,)


...

C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


(34249, 1) (34249,) (12003, 1) (12003,)


C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


(51865, 1) (51865,) (7465, 1) (7465,)
FINAL: k=17, under5_mort     r^2 = 0.00
=========== asset_index     ============


C:\Users\User\AppData\Local\Temp\ipykernel_27048\118561935.py:42: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()
